In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Data/rules/MNXreaction_smiles_atommap_rules_filter.csv'
target_smiles_path = '../../../Results/not_lipid/target_smiles_not_lipid_complex.pickle'
target_model_smiles_path = '../../../Results/not_lipid/yeast_met_not_lipid.pickle'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Data/analysis/total_met_inchikey0.pickle'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 70
num_process = 50
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}/'
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid/top{num}_{cut_off}/'

sce_gene_clean_ec = '../../../Data/Saccharomyces_cerevisiae_maxsep.csv'
DeepEC_path = '../../../Data/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/uniprotkb_organism_id_559292_2023_11_08.tsv'
yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'

###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/not_lipid/retrosys_smiles_calculate_similarity_{cut_off}_re.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_uptake_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'




In [3]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(cut_off_retrosynthesis_path):
    os.makedirs(cut_off_retrosynthesis_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [4]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')


total_smiles 3680
total_smiles 3679
total_smiles 3679
total_smiles 2222


21918it [00:01, 11514.62it/s]


8671


21918it [00:02, 9002.87it/s] 

12278


In [5]:
# retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
# retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
# retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

In [6]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)

# choose top num similar metabolites
retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

100%|██████████| 8671/8671 [12:55<00:00, 11.17it/s]


### get rxndb

In [7]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)

100%|██████████| 21918/21918 [01:13<00:00, 300.19it/s]


In [8]:
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 50/50 [00:09<00:00,  5.07it/s]


In [9]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)

 23%|██▎       | 4955/21918 [00:23<02:26, 115.95it/s]

100%|██████████| 21918/21918 [04:35<00:00, 79.61it/s] 


### retrosynthesis result screen

In [12]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 21918/21918 [00:04<00:00, 4662.09it/s]


rxndb: 4375000
rxndb_drop: 92907


In [13]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(92907, 9)


In [4]:
# #merge the rxndb
# # rxndb = merge_rxndb(rxndb_path,rxndb_all_path)
with open(rxndb_all_path,'r') as f:
    rxndb = json.load(f)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(92907, 9)


In [5]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)

100%|██████████| 100/100 [00:00<00:00, 272.61it/s]


number of metabolites in RXNDB: 4707


In [6]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)

In [7]:
#calculate the similarity of the metabolites

smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)

100%|██████████| 4707/4707 [08:44<00:00,  8.98it/s]


In [10]:
# smile_max_score_pd = pd.read_csv(rxndb_met_max_score_file)

In [9]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

100%|██████████| 100/100 [00:03<00:00, 28.03it/s]


Iteration 1 - Current success count: 4023
Iteration 1 - Current success_rxndbid count: 66278


100%|██████████| 100/100 [00:03<00:00, 27.70it/s]


Iteration 2 - Current success count: 4253
Iteration 2 - Current success_rxndbid count: 70312


100%|██████████| 100/100 [00:03<00:00, 27.55it/s]


...

Iteration 5 - Current success count: 4304
Iteration 5 - Current success_rxndbid count: 72138


100%|██████████| 100/100 [00:03<00:00, 26.24it/s]


Iteration 6 - Current success count: 4311
Iteration 6 - Current success_rxndbid count: 72229


100%|██████████| 100/100 [00:04<00:00, 24.90it/s]


Iteration 7 - Current success count: 4311
Iteration 7 - Current success_rxndbid count: 72229
final success 4311
final success_rxndbid 72229


In [ ]:
# smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smiles_max_score_tmp_pd, num_processes=num_process, num_iterations=2)
# smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

In [ ]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_file)

In [11]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)

success number: 215
fail number: 360


In [ ]:
# smiles_success = load_pickle(YMDB_success_met_smile_file)
# inchikey0_success = [smiles2inchikey0(x) for x in smiles_success]
# print(len(uptake_smiles))
# uptake_drop = [x for x in uptake_smiles if smiles2inchikey0(x) not in inchikey0_success ]
# print(len(uptake_drop))

63
51


In [ ]:
# uptake_smiles_df = pd.read_csv(uptake_met_path)

# uptake_smiles_df['inchikey0'] = uptake_smiles_df['SMILES'].progress_apply(smiles2inchikey0)
# uptake_smiles_df = uptake_smiles_df[~uptake_smiles_df['inchikey0'].isin(inchikey0_success)]
# uptake_smiles_df.to_csv(uptake_met_path, index=False)
# len(uptake_smiles_df)

100%|██████████| 63/63 [00:00<00:00, 1751.50it/s]


51

In [12]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)

# consider uptake metabolites
uptake = pd.read_csv(uptake_met_path)
uptake_smiles = uptake['SMILES'].values
uptake_smiles = [normalize_smiles(i) for i in uptake_smiles]
yeast8_total_smiles = yeast8_total_smiles + uptake_smiles

In [13]:
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_uptake_file)

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_uptake_file,index=None)

100%|██████████| 100/100 [00:03<00:00, 29.17it/s]


Iteration 1 - Current success count: 4236
Iteration 1 - Current success_rxndbid count: 81932


100%|██████████| 100/100 [00:03<00:00, 26.55it/s]


Iteration 2 - Current success count: 4474
Iteration 2 - Current success_rxndbid count: 86456


100%|██████████| 100/100 [00:03<00:00, 26.73it/s]


...

Iteration 5 - Current success count: 4523
Iteration 5 - Current success_rxndbid count: 87977


100%|██████████| 100/100 [00:04<00:00, 24.51it/s]


Iteration 6 - Current success count: 4529
Iteration 6 - Current success_rxndbid count: 88065


100%|██████████| 100/100 [00:03<00:00, 26.02it/s]


Iteration 7 - Current success count: 4529
Iteration 7 - Current success_rxndbid count: 88065
final success 4529
final success_rxndbid 88065


In [14]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_uptake_file,YMDB_fail_met_smile_uptake_file)

success number: 254
fail number: 321


In [15]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)

(88065, 10)
(88065, 10)
(88065, 5)
(88065, 5)


In [16]:
smiles_success = load_pickle(YMDB_success_met_smile_file)
smiles_success_uptake = load_pickle(YMDB_success_met_smile_uptake_file)
diff = [x for x in smiles_success_uptake if x not in smiles_success]
uptake_synthesis = [x for x in diff if x not in uptake_smiles]
print(len(uptake_synthesis))

4
